In [186]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import time
from config import api_key
import pprint as pp
from collections import Counter
import math
%matplotlib inline

# The Movie Database - API Calls

In [188]:
#url string to pass to requests
base_url = 'https://api.themoviedb.org/3/discover/movie?api_key='

url = f"{base_url}{api_key}"

#parameters (queries) to pass to API Call
language = 'en-US'
region = 'US'
sort_by = 'revenue.desc'
primary_release_year = None
_release_date = '1989-01-01'

#function to make a single API call for page 'p'

def make_request(page_num):
    
    payload = {'language': language,
               'region': region,
               'sort_by': sort_by,
               'primary_release_year': primary_release_year,
               'primary_release_date.gte': _release_date,
               'page': page_num
          }
    
    r = requests.get(url, params=payload)
    print(f"page number {page_num} search completed")
    
    return r.json()

In [189]:
# function to make 'n' api calls
def make_api_calls(start_index, end_index):
    
    api_results = []
    
    for page_num in range(start_index, end_index + 1):
        api_results.append(make_request(page_num))
        time.sleep(1)
        
    return api_results

In [190]:
# function to add all movie ids from your disover query to a list
def extract_ids(data):
    
    list_ids = []
    
    for page in data:
        
        for result in page['results']:
            list_ids.append(result['id'])

    return list_ids

In [192]:
# function to query single film using list of ids
def query_movie(list_ids):
    
    query_url = 'https://api.themoviedb.org/3/movie/'
    
    q_results = []
    
    payload = {}
    
    for _id in list_ids:
        
        r = requests.get(f'{query_url}{_id}?api_key={api_key}', params=payload)
        q_results.append(r.json())
        print(f"movie {_id} requested, {len(q_results)} total films in this query")
        time.sleep(0.25)
    
    return q_results

In [193]:
#function that brings all previous api functions together to look for 'n' total results
_results_per_page = 20
_max_pages_per_loop = 50

def make_queries(total_results):
    
    pages = math.ceil(total_results / _results_per_page) 
    loops = max(1, math.ceil(pages / _max_pages_per_loop)) 
    start_index = 1
    end_index = max(1, min(pages, _max_pages_per_loop)) 
    
    mq_output = []
    
    for i in range(1, loops + 1):
        
        calls = make_api_calls(start_index, end_index)
        list_ids =  extract_ids(calls)
        qm_output = query_movie(list_ids)
        
        for movie in qm_output:
            mq_output.append(movie)
            
        start_index, end_index = (end_index + 1), ((i + 1) * _max_pages_per_loop)
        print(f"------> loop {i} complete, {len(mq_output)} total movies in list")
        time.sleep(30)
        
    df = pd.DataFrame(mq_output)
    
    return df

df = make_queries(5000)
df.shape

page number 1 search completed
page number 2 search completed
page number 3 search completed
page number 4 search completed
page number 5 search completed
page number 6 search completed
page number 7 search completed
page number 8 search completed
page number 9 search completed
page number 10 search completed
page number 11 search completed
page number 12 search completed
page number 13 search completed
page number 14 search completed
page number 15 search completed
page number 16 search completed
page number 17 search completed
page number 18 search completed
page number 19 search completed
page number 20 search completed
page number 21 search completed
page number 22 search completed
page number 23 search completed
page number 24 search completed
page number 25 search completed
page number 26 search completed
page number 27 search completed
page number 28 search completed
page number 29 search completed
page number 30 search completed
page number 31 search completed
page number 32 se

movie 9502 requested, 128 total films in this query
movie 9806 requested, 129 total films in this query
movie 286217 requested, 130 total films in this query
movie 51497 requested, 131 total films in this query
movie 8960 requested, 132 total films in this query
movie 41154 requested, 133 total films in this query
movie 10138 requested, 134 total films in this query
movie 2062 requested, 135 total films in this query
movie 363088 requested, 136 total films in this query
movie 330 requested, 137 total films in this query
movie 263115 requested, 138 total films in this query
movie 615 requested, 139 total films in this query
movie 11631 requested, 140 total films in this query
movie 82702 requested, 141 total films in this query
movie 87827 requested, 142 total films in this query
movie 335988 requested, 143 total films in this query
movie 10527 requested, 144 total films in this query
movie 36557 requested, 145 total films in this query
movie 38757 requested, 146 total films in this que

movie 341174 requested, 284 total films in this query
movie 12405 requested, 285 total films in this query
movie 10020 requested, 286 total films in this query
movie 82675 requested, 287 total films in this query
movie 166428 requested, 288 total films in this query
movie 272 requested, 289 total films in this query
movie 3981 requested, 290 total films in this query
movie 862 requested, 291 total films in this query
movie 270946 requested, 292 total films in this query
movie 2080 requested, 293 total films in this query
movie 2268 requested, 294 total films in this query
movie 17578 requested, 295 total films in this query
movie 72559 requested, 296 total films in this query
movie 534 requested, 297 total films in this query
movie 1771 requested, 298 total films in this query
movie 137113 requested, 299 total films in this query
movie 544 requested, 300 total films in this query
movie 210577 requested, 301 total films in this query
movie 5503 requested, 302 total films in this query
m

movie 27576 requested, 440 total films in this query
movie 19585 requested, 441 total films in this query
movie 447200 requested, 442 total films in this query
movie 2655 requested, 443 total films in this query
movie 268896 requested, 444 total films in this query
movie 1422 requested, 445 total films in this query
movie 1934 requested, 446 total films in this query
movie 1979 requested, 447 total films in this query
movie 157350 requested, 448 total films in this query
movie 2502 requested, 449 total films in this query
movie 55721 requested, 450 total films in this query
movie 37786 requested, 451 total films in this query
movie 8839 requested, 452 total films in this query
movie 5559 requested, 453 total films in this query
movie 2770 requested, 454 total films in this query
movie 254470 requested, 455 total films in this query
movie 75612 requested, 456 total films in this query
movie 68724 requested, 457 total films in this query
movie 944 requested, 458 total films in this query

movie 339403 requested, 596 total films in this query
movie 37958 requested, 597 total films in this query
movie 8681 requested, 598 total films in this query
movie 32657 requested, 599 total films in this query
movie 10201 requested, 600 total films in this query
movie 489 requested, 601 total films in this query
movie 339964 requested, 602 total films in this query
movie 37799 requested, 603 total films in this query
movie 7443 requested, 604 total films in this query
movie 60304 requested, 605 total films in this query
movie 457251 requested, 606 total films in this query
movie 1701 requested, 607 total films in this query
movie 628 requested, 608 total films in this query
movie 9315 requested, 609 total films in this query
movie 225574 requested, 610 total films in this query
movie 9354 requested, 611 total films in this query
movie 8247 requested, 612 total films in this query
movie 8487 requested, 613 total films in this query
movie 274855 requested, 614 total films in this query

...

movie 157832 requested, 516 total films in this query
movie 9771 requested, 517 total films in this query
movie 24662 requested, 518 total films in this query
movie 129139 requested, 519 total films in this query
movie 25674 requested, 520 total films in this query
movie 60420 requested, 521 total films in this query
movie 50837 requested, 522 total films in this query
movie 842 requested, 523 total films in this query
movie 10185 requested, 524 total films in this query
movie 32855 requested, 525 total films in this query
movie 1392 requested, 526 total films in this query
movie 14313 requested, 527 total films in this query
movie 1543 requested, 528 total films in this query
movie 269795 requested, 529 total films in this query
movie 265177 requested, 530 total films in this query
movie 177494 requested, 531 total films in this query
movie 397262 requested, 532 total films in this query
movie 32141 requested, 533 total films in this query
movie 53116 requested, 534 total films in thi

movie 10832 requested, 670 total films in this query
movie 13637 requested, 671 total films in this query
movie 10360 requested, 672 total films in this query
movie 56836 requested, 673 total films in this query
movie 29888 requested, 674 total films in this query
movie 340 requested, 675 total films in this query
movie 19124 requested, 676 total films in this query
movie 68351 requested, 677 total films in this query
movie 214030 requested, 678 total films in this query
movie 10548 requested, 679 total films in this query
movie 12616 requested, 680 total films in this query
movie 338928 requested, 681 total films in this query
movie 199160 requested, 682 total films in this query
movie 9950 requested, 683 total films in this query
movie 228968 requested, 684 total films in this query
movie 180299 requested, 685 total films in this query
movie 14125 requested, 686 total films in this query
movie 50797 requested, 687 total films in this query
movie 45658 requested, 688 total films in th

movie 13256 requested, 824 total films in this query
movie 429200 requested, 825 total films in this query
movie 20794 requested, 826 total films in this query
movie 14631 requested, 827 total films in this query
movie 17539 requested, 828 total films in this query
movie 12143 requested, 829 total films in this query
movie 339 requested, 830 total films in this query
movie 64483 requested, 831 total films in this query
movie 493099 requested, 832 total films in this query
movie 24230 requested, 833 total films in this query
movie 409502 requested, 834 total films in this query
movie 377154 requested, 835 total films in this query
movie 310593 requested, 836 total films in this query
movie 21625 requested, 837 total films in this query
movie 31458 requested, 838 total films in this query
movie 13944 requested, 839 total films in this query
movie 16027 requested, 840 total films in this query
movie 69523 requested, 841 total films in this query
movie 1600 requested, 842 total films in th

movie 401246 requested, 979 total films in this query
movie 32074 requested, 980 total films in this query
movie 311777 requested, 981 total films in this query
movie 10938 requested, 982 total films in this query
movie 13646 requested, 983 total films in this query
movie 348540 requested, 984 total films in this query
movie 1413 requested, 985 total films in this query
movie 31651 requested, 986 total films in this query
movie 15483 requested, 987 total films in this query
movie 9662 requested, 988 total films in this query
movie 250124 requested, 989 total films in this query
movie 292040 requested, 990 total films in this query
movie 11633 requested, 991 total films in this query
movie 55283 requested, 992 total films in this query
movie 24927 requested, 993 total films in this query
movie 228674 requested, 994 total films in this query
movie 33379 requested, 995 total films in this query
movie 29047 requested, 996 total films in this query
movie 12901 requested, 997 total films in 

(5000, 25)

In [194]:
df.head(10)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/aHcth2AXzZSjhX7JYh7ie73YVNc.jpg,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,...,2009-12-10,2787965087,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.4,18093
1,False,/njv65RTipNSTozFLuF85jL0bcQe.jpg,"{'id': 10, 'name': 'Star Wars Collection', 'po...",245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,en,Star Wars: The Force Awakens,...,2015-12-15,2068223624,136.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every generation has a story.,Star Wars: The Force Awakens,False,7.4,12292
2,False,/bOGkgRGdhrBYJSLpXaxhXVstddV.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://marvel.com/movies/movie/223/avengers_in...,299536,tt4154756,en,Avengers: Infinity War,...,2018-04-25,2046239637,149.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An entire universe. Once and for all.,Avengers: Infinity War,False,8.3,12368
3,False,/vFUI5obFtx4IdhP6k8Om5ezHTrk.jpg,None,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",None,597,tt0120338,en,Titanic,...,1997-11-18,1845034188,194.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Nothing on Earth could come between them.,Titanic,False,7.8,13660
4,False,/t5KONotASgVKq4N19RyhIthWOPG.jpg,"{'id': 328, 'name': 'Jurassic Park Collection'...",150000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.jurassicworld.com/,135397,tt0369610,en,Jurassic World,...,2015-06-06,1671713208,124.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The park is open.,Jurassic World,False,6.6,13661
5,False,/hbn46fQaRmlpBuUrEiFqv0GDL6Y.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://marvel.com/avengers_movie/,24428,tt0848228,en,The Avengers,...,2012-04-25,1519557910,143.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Some assembly required.,The Avengers,False,7.6,18600
6,False,/ypyeMfKydpyuuTMdp36rMlkGDUL.jpg,"{'id': 9485, 'name': 'The Fast and the Furious...",190000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.furious7.com/,168259,tt2820852,en,Furious 7,...,2015-04-01,1506249360,137.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Vengeance Hits Home,Furious 7,False,7.3,6322
7,False,/rFtsE7Lhlc2jRWF7SRAU0fvrveQ.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",280000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://marvel.com/movies/movie/193/avengers_ag...,99861,tt2395427,en,Avengers: Age of Ultron,...,2015-04-22,1405403694,141.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A New Age Has Come.,Avengers: Age of Ultron,False,7.3,12563
8,False,/6ELJEzQJ3Y45HczvreC3dg0GV5R.jpg,"{'id': 529892, 'name': 'Black Panther Collecti...",200000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://marvel.com/movies/movie/224/black_panther,284054,tt1825683,en,Black Panther,...,2018-02-13,1346739107,134.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Long live the king.,Black Panther,False,7.4,11386
9,False,/n5A7brJCjejceZmHyujwUTVgQNC.jpg,"{'id': 1241, 'name': 'Harry Potter Collection'...",125000000,"[{'id': 10751, 'name': 'Family'}, {'id': 14, '...",http://hpwishlist.warnerbros.com/,12445,tt1201607,en,Harry Potter and the Deathly Hallows: Part 2,...,2011-07-07,1342000000,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,It all ends here.,Harry Potter and the Deathly Hallows: Part 2,False,8.1,11077


In [140]:
#Drop Unnecessary Columns
df = df.drop(columns=['adult', 'backdrop_path', 'original_title', 'belongs_to_collection', 'homepage','poster_path', 'spoken_languages', 'status', 'video'])

In [141]:
#Rearrange the Columns
df = df[['id', 'imdb_id', 'title', 'release_date', 'genres',
         'revenue', 'budget', 'runtime', 'tagline', 'overview', 'production_companies', 'production_countries', 'vote_average', 'vote_count' ]]

In [142]:
#Cast release_date as datetime
df['release_date'] = pd.to_datetime(df['release_date'])

In [143]:
#Make a Copy of the DataFrame to work on
dfx = df.copy(deep=True)
dfx.shape
dfx.head()

(1000, 14)

,id,imdb_id,title,release_date,genres,revenue,budget,runtime,tagline,overview,production_companies,production_countries,vote_average,vote_count
0,19995,tt0499549,Avatar,2009-12-10,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",2787965087,237000000,162,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...","[{'id': 289, 'logo_path': None, 'name': 'Ingen...","[{'iso_3166_1': 'US', 'name': 'United States o...",7.4,18090
1,140607,tt2488496,Star Wars: The Force Awakens,2015-12-15,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",2068223624,245000000,136,Every generation has a story.,Thirty years after defeating the Galactic Empi...,"[{'id': 1634, 'logo_path': None, 'name': 'True...","[{'iso_3166_1': 'US', 'name': 'United States o...",7.4,12289
2,299536,tt4154756,Avengers: Infinity War,2018-04-25,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",2046239637,300000000,149,An entire universe. Once and for all.,As the Avengers and their allies have continue...,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",8.3,12362
3,597,tt0120338,Titanic,1997-11-18,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",1845034188,200000000,194,Nothing on Earth could come between them.,101-year-old Rose DeWitt Bukater tells the sto...,"[{'id': 4, 'logo_path': '/fycMZt242LVjagMByZOL...","[{'iso_3166_1': 'US', 'name': 'United States o...",7.8,13657
4,135397,tt0369610,Jurassic World,2015-06-06,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1671713208,150000000,124,The park is open.,Twenty-two years after the events of Jurassic ...,"[{'id': 3341, 'logo_path': '/dTG5dXE1kU2mpmL9B...","[{'iso_3166_1': 'US', 'name': 'United States o...",6.6,13660


In [775]:
list_of_lists = []

for i in range(len(dfx)):
    list_cos = []
    for j in dfx.iloc[i]['production_companies']:
        list_cos.append(j['name'])
    list_of_lists.append(list_cos)
    
dfx['all_companies'] = list_of_lists

In [778]:
#Make CSV File from Dataframe (from API Call)
dfx.to_csv('dfx_movies.csv')

# The Numbers - Web Scraping

In [170]:
#Scrape The-Numbers Website
from bs4 import BeautifulSoup

def build_url(page):
    host = 'https://www.the-numbers.com/movie/budgets/all/'
    return f"{host}{page}"

def scrape(page):
    #calls url function and parses results into Beautiful Soup
    r = requests.get(build_url(page))
    c = r.content
    soup = BeautifulSoup(c, 'html.parser')
    return soup


In [171]:
def parse_scrape(soup_object):

    table = soup_object.find( "table")
    movie_data = table.find_all('td', class_ = 'data')

    titles = []
    movie_list = []
    
    for i in table.find_all('b'):
        titles.append(i.get_text())

    i = 0
    for row in range(int(len(movie_data)/4)):
        movie_list.append({'name': titles[row], 'budget': movie_data[row+i+1].get_text(), 'dom_gross': movie_data[row+i+2].get_text(), 'ww_gross': movie_data[row+i+3].get_text()})
        i += 3
        
    return movie_list

def loop_through(loops):
    
    movie_list = []
    
    adder = 100
    page = 1
    
    for i in range(loops):
        output = parse_scrape(scrape(page))
        for j in range(len(output)):
            movie_list.append(output[j])
        page += adder
        
    return movie_list

movie = loop_through(20)

In [178]:
from pandas.io.json import json_normalize
data = movie
movie_nums = json_normalize(data)
movie_nums.head()

,budget,dom_gross,name,ww_gross
0,"$425,000,000","$760,507,625",Avatar,"$2,776,345,279"
1,"$410,600,000","$241,063,875",Pirates of the Caribbean: On Stranger Tides,"$1,045,663,875"
2,"$330,600,000","$459,005,868",Avengers: Age of Ultron,"$1,403,013,963"
3,"$317,000,000","$620,181,382",Star Wars Ep. VIII: The Last Jedi,"$1,316,721,747"
4,"$306,000,000","$936,662,225",Star Wars Ep. VII: The Force Awakens,"$2,053,311,220"


In [179]:
movie_nums = movie_nums[['name', 'budget', 'dom_gross', 'ww_gross']]
movie_nums.head()

,name,budget,dom_gross,ww_gross
0,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
3,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
4,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220"
